In [15]:
import os
import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def exposure_loss(x, mean_val=0.6):
    x = tf.reduce_mean(x, axis=3, keepdims=True)
    mean = tf.nn.avg_pool2d(x, ksize=16, strides=16, padding="VALID")
    return tf.reduce_mean(tf.square(mean - mean_val))

def calculate_illumination_smoothness_loss(image_tensor):

    batch_size = tf.shape(image_tensor)[0]
    height = tf.shape(image_tensor)[1]
    width = tf.shape(image_tensor)[2]

    height_elements = (tf.shape(image_tensor)[2] - 1) * tf.shape(image_tensor)[3]
    width_elements = tf.shape(image_tensor)[2] * (tf.shape(image_tensor)[3] - 1)

    height_tv = tf.reduce_sum(tf.square(image_tensor[:, 1:, :, :] - image_tensor[:, :height - 1, :, :]))
    width_tv = tf.reduce_sum(tf.square(image_tensor[:, :, 1:, :] - image_tensor[:, :, :width - 1, :]))

    batch_size = tf.cast(batch_size, dtype=tf.float32)
    height_elements = tf.cast(height_elements, dtype=tf.float32)
    width_elements = tf.cast(width_elements, dtype=tf.float32)

    return 2 * (height_tv / height_elements + width_tv / width_elements) / batch_size


def compute_color_consistency_loss(image):

    mean_values = tf.reduce_mean(image, axis=(1, 2), keepdims=True)
    red_mean, green_mean, blue_mean = mean_values[:, :, :, 0], mean_values[:, :, :, 1], mean_values[:, :, :, 2]

    diff_red_green = tf.square(red_mean - green_mean)
    diff_red_blue = tf.square(red_mean - blue_mean)
    diff_green_blue = tf.square(blue_mean - green_mean)

    return tf.sqrt(diff_red_green + diff_red_blue + diff_green_blue)


def build_custom_dce_net():

    input_img = keras.Input(shape=[None, None, 3])

    conv1 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(input_img)
    conv2 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv1)
    conv3 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv2)
    conv4 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(conv3)

    int_con1 = layers.Concatenate(axis=-1)([conv4, conv3])
    conv5 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(int_con1)
    int_con2 = layers.Concatenate(axis=-1)([conv5, conv2])
    conv6 = layers.Conv2D(32, (3, 3), strides=(1, 1), activation="relu", padding="same")(int_con2)
    int_con3 = layers.Concatenate(axis=-1)([conv6, conv1])

    x_r = layers.Conv2D(24, (3, 3), strides=(1, 1), activation="tanh", padding="same")(int_con3)

    model = keras.Model(inputs=input_img, outputs=x_r)
    return model

class ConsistencyLoss(keras.losses.Loss):
    def __init__(self, **kwargs):
        super(ConsistencyLoss, self).__init__(reduction="none")

        self.kernel_left = tf.constant(
            [[[[0, 0, 0]], [[-1, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.kernel_right = tf.constant(
            [[[[0, 0, 0]], [[0, 1, -1]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.kernel_up = tf.constant(
            [[[[0, -1, 0]], [[0, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.kernel_down = tf.constant(
            [[[[0, 0, 0]], [[0, 1, 0]], [[0, -1, 0]]]], dtype=tf.float32
        )

    def call(self, true_values, predicted_values):

        mean_true = tf.reduce_mean(true_values, axis=3, keepdims=True)
        mean_predicted = tf.reduce_mean(predicted_values, axis=3, keepdims=True)


        pooled_true = tf.nn.avg_pool2d(
            mean_true, ksize=4, strides=4, padding="VALID"
        )
        pooled_predicted = tf.nn.avg_pool2d(
            mean_predicted, ksize=4, strides=4, padding="VALID"
        )

        grad_true_left = tf.nn.conv2d(
            pooled_true, self.kernel_left, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_true_right = tf.nn.conv2d(
            pooled_true, self.kernel_right, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_true_up = tf.nn.conv2d(
            pooled_true, self.kernel_up, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_true_down = tf.nn.conv2d(
            pooled_true, self.kernel_down, strides=[1, 1, 1, 1], padding="SAME"
        )

        grad_pred_left = tf.nn.conv2d(
            pooled_predicted, self.kernel_left, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_pred_right = tf.nn.conv2d(
            pooled_predicted, self.kernel_right, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_pred_up = tf.nn.conv2d(
            pooled_predicted, self.kernel_up, strides=[1, 1, 1, 1], padding="SAME"
        )
        grad_pred_down = tf.nn.conv2d(
            pooled_predicted, self.kernel_down, strides=[1, 1, 1, 1], padding="SAME"
        )

        diff_left = tf.square(grad_true_left - grad_pred_left)
        diff_right = tf.square(grad_true_right - grad_pred_right)
        diff_up = tf.square(grad_true_up - grad_pred_up)
        diff_down = tf.square(grad_true_down - grad_pred_down)

        return diff_left + diff_right + diff_up + diff_down

class ZeroDCE(keras.Model):
    def __init__(self, **kwargs):
        super(ZeroDCE, self).__init__(**kwargs)
        self.dce_model = build_custom_dce_net()

    def compile(self, learning_rate, **kwargs):
        super(ZeroDCE, self).compile(**kwargs)
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.constancy_loss = ConsistencyLoss(reduction="none")

    def get_enhanced_image(self, data, output):
        r1 = output[:, :, :, :3]
        r2 = output[:, :, :, 3:6]
        r3 = output[:, :, :, 6:9]
        r4 = output[:, :, :, 9:12]
        r5 = output[:, :, :, 12:15]
        r6 = output[:, :, :, 15:18]
        r7 = output[:, :, :, 18:21]
        r8 = output[:, :, :, 21:24]
        x = data + r1 * (tf.square(data) - data)
        x = x + r2 * (tf.square(x) - x)
        x = x + r3 * (tf.square(x) - x)
        enhanced_image = x + r4 * (tf.square(x) - x)
        x = enhanced_image + r5 * (tf.square(enhanced_image) - enhanced_image)
        x = x + r6 * (tf.square(x) - x)
        x = x + r7 * (tf.square(x) - x)
        enhanced_image = x + r8 * (tf.square(x) - x)
        return enhanced_image

    def call(self, data):
        dce_net_output = self.dce_model(data)
        return self.get_enhanced_image(data, dce_net_output)

    def compute_losses(self, data, output):
        enhanced_image = self.get_enhanced_image(data, output)
        loss_illumination = 200 * calculate_illumination_smoothness_loss(output)
        loss_spatial_constancy = tf.reduce_mean(
            self.constancy_loss(enhanced_image, data)
        )
        loss_color_constancy = 5 * tf.reduce_mean(compute_color_consistency_loss(enhanced_image))
        loss_exposure = 10 * tf.reduce_mean(exposure_loss(enhanced_image))
        total_loss = (
            loss_illumination
            + loss_spatial_constancy
            + loss_color_constancy
            + loss_exposure
        )
        return {
            "total_loss": total_loss,
            "illumination_smoothness_loss": loss_illumination,
            "spatial_constancy_loss": loss_spatial_constancy,
            "color_constancy_loss": loss_color_constancy,
            "exposure_loss": loss_exposure,
        }

    def train_step(self, data):
        with tf.GradientTape() as tape:
            output = self.dce_model(data)
            losses = self.compute_losses(data, output)
        gradients = tape.gradient(
            losses["total_loss"], self.dce_model.trainable_weights
        )
        self.optimizer.apply_gradients(zip(gradients, self.dce_model.trainable_weights))
        return losses

    def test_step(self, data):
        output = self.dce_model(data)
        return self.compute_losses(data, output)

    def save_weights(self, filepath, overwrite=True, save_format=None, options=None):
        """While saving the weights, we simply save the weights of the DCE-Net"""
        self.dce_model.save_weights(
            filepath, overwrite=overwrite, save_format=save_format, options=options
        )

    def load_weights(self, filepath, by_name=False, skip_mismatch=False, options=None):
        """While loading the weights, we simply load the weights of the DCE-Net"""
        self.dce_model.load_weights(
            filepath=filepath,
            by_name=by_name,
            skip_mismatch=skip_mismatch,
            options=options,
        )

def infer(original_image):
    image = keras.preprocessing.image.img_to_array(original_image)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    output_image = zero_dce_model(image)
    output_image = tf.cast((output_image[0, :, :, :] * 255), dtype=np.uint8)
    output_image = Image.fromarray(output_image.numpy())
    return output_image

zero_dce_model = ZeroDCE()
#os.chdir('/content/drive/MyDrive')
current_directory = os.getcwd()
weights=r'zerodcemodel.h5'
model_path=os.path.join(current_directory,weights)
zero_dce_model.load_weights(model_path)

test_imgs = sorted(glob('./test/low/*'))
save_dir = r'test/predicted'
current_directory = os.getcwd()
path = os.path.join(current_directory,save_dir)

if not os.path.exists(path):
    os.makedirs(path)




for val_image_file in test_imgs:

    original_image = Image.open(val_image_file)
    enhanced_image = infer(original_image)

    original_image_np = np.array(original_image)
    enhanced_image_np = np.array(enhanced_image)

    file_name = os.path.basename(val_image_file)
    output_path = os.path.join(path, 'enhanced_' + file_name)
    enhanced_image.save(output_path)

    plt.imshow(enhanced_image)
    plt.axis('off')
    plt.show()